In [1]:
# !pip3 install pandas
# !pip3 install scikit-learn
# !pip3 install optuna
# !pip3 install --upgrade wandb

In [2]:
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import time

# Neural Network Components
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

#Data preprocessing
from sklearn.model_selection import train_test_split
from PIL import Image

# Experiments Managers
import optuna
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

2025-11-02 01:43:43.740263: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-02 01:43:43.766318: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-02 01:43:45.386707: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
# wandb.login()

In [4]:
#Import the modules
from complete_model_creator import NeuralNetworkConstructor
from complete_model_creator import wandb_block
from complete_model_creator import ModelBuilder
from complete_model_creator import  ImageDataGenerator

In [5]:
csv_directory = pd.read_csv('/tf/keras_neural_network/Mis_Tests/plant-pathology-2020-fgvc7/train.csv')
csv_directory.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [6]:
image_id_column = 'image_id'
image_label_column = ['healthy', 'multiple_diseases', 'rust', 'scab']
number_of_classes =len(image_label_column)

In [7]:
train_df, temp_df = train_test_split(csv_directory, 
                                     test_size=0.4, 
                                     random_state=42, 
                                     stratify=csv_directory[image_label_column].idxmax(axis=1))
test_df, val_df = train_test_split(temp_df, 
                                   test_size=0.25, 
                                   random_state=42, 
                                   stratify=temp_df[image_label_column].idxmax(axis=1))

In [8]:
image_directory = '/tf/keras_neural_network/Mis_Tests/plant-pathology-2020-fgvc7/images/'
batch_size = 25
inputs = (2048,1365,3)
image_size = inputs[:2]
train_generator = ImageDataGenerator(train_df, image_directory, batch_size, image_size)
val_generator = ImageDataGenerator(val_df, image_directory, batch_size, image_size, shuffle=False)
test_generator = ImageDataGenerator(test_df, image_directory, batch_size, image_size, shuffle=False)

In [11]:
def objective(trial):
    # Early stopping callback
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience=50, 
        restore_best_weights=True,
        mode="auto",
        verbose=1,
        baseline=None
    )

    builder = ModelBuilder(
        trial, 
        inputs=keras.Input(inputs),
        total_classes=number_of_classes+2 
    )
    
    model = builder.get_model()
    params = builder.get_params(2)
    
    run = wandb_block(params)
    
    # Train the model with data generators
    history = model.fit(
        train_generator,
        epochs=1000, 
        batch_size=batch_size,  # Use the tuned batch_size
        verbose=0,
        validation_data=val_generator,
        callbacks=[
            WandbMetricsLogger(log_freq=builder.sampling_interval),
            early_stopping
        ]
    )
    
    # Finish wandb run
    run.finish()
    
    # Evaluate on validation set (not test set for hyperparameter optimization)
    val_loss, val_accuracy = model.evaluate(val_generator, verbose=0)
    
    # Optional: Also evaluate on test set for final reporting
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
    
    print(f"Trial {trial.number} - Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    print(f"Trial {trial.number} - Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")
    
    return val_loss  # Optimize on validation loss

In [12]:
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 30)

[I 2025-11-02 01:45:59,985] A new study created in memory with name: no-name-166964c8-11a6-44b5-a306-679cef676913


/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
[W 2025-11-02 01:46:02,881] Trial 0 failed with parameters: {'conv_blocks': 2, 'maxpooling_tate': 5, 'bottleneck_length': 1, 'filters': 128, 'bottleneck_dropout': 0.25009582801029007, 'optimizer': 'SGD', 'BN_activation0': 'relu', 'BN_regularizer0': 'L2', 'DW_activation0': 'mish', 'DW_regularizer0': 'L2', 'UW_activation0': 'silu', 'UW_regularizer0': None, 'DW_activation1': 'relu', 'DW_regularizer1': 'L1L2', 'UW_activation1': 'mish', 'UW_regularizer1': None, 'DW_activation2': 'elu', 'DW_regularizer2': 'L1L2', 'UW_activation2': 'tanh', 'UW_regularizer2': 'L1', 'DW_activation3': 'silu', 'DW_regularizer3': 'L1L2', 'UW_activat

NameError: name 'label_columns' is not defined